this notebook is based on my understanding of chapter-15 of 'The Quick Python Book' by Naomi Ceder.

## inheritance

* if more than one class use same fields or methods, we can abstract them out into a new class and inherit them whenever required.

In [105]:
class Shape:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def move(self, delta_x, delta_y):
        self.x = self.x + delta_x
        self.y = self.y + delta_y
    
class Square(Shape):
    def __init__(self, side=1, x=0, y=0):
        super().__init__(x, y) # one way of calling super class's __init__
        self.side = side
    
class Circle(Shape):
    def __init__(self, r=1, x=0, y=0):
        # hardcoding the class name creates problem later and is less flexible
        Shape.__init__(self, x, y) # other less-used way of calling super class's __init__
        self.radius = r

In [106]:
c = Circle(1)
print(c.x)
print(c.y)

0
0


**python methods are virtual**. unless overriden by child class, super class methods are used.

In [107]:
c.move(3, 4) # 'move' method is in Shape which is inherited by Circle
print(c.x)
print(c.y)

3
4


#### exercise

In [108]:
class Rectangle(Shape):
    def __init__(self, l=1, w=2, x=0, y=0):
        super().__init__(x, y)
        self.l = l
        self.w = w
    
    def area(self):
        return self.l * self.w

In [109]:
r = Rectangle(2, 3)
print(r.x, r.y)
print(r.l, r.w)

0 0
2 3


In [110]:
r.move(3, 4)
print(r.x, r.y)

3 4


In [111]:
r.area()

6

create a `Square` class which inherits `Rectangle`

In [112]:
class Square(Rectangle):
    def __init__(self, l=1, x=0, y=0):
        super().__init__(l, l, x, y)
        self.l = l

In [113]:
s = Square(5)
print(s.l)

5


In [114]:
print(s.area())

25


***is super().__init__() always necessary?***

In [115]:
class Dummy(Shape):
    pass

c = Dummy()

TypeError: __init__() missing 2 required positional arguments: 'x' and 'y'

In [116]:
class Dummy(Shape):
    def __init__(self):
        pass

c = Dummy()

In [117]:
c.x, c.y

AttributeError: 'Dummy' object has no attribute 'x'

so it is not necessary to call `super().__init__()` always.

### inheritance with class and instance variables
* inheritance allows an instance to inherit attributes of the class.
* only one instance variable of a given name exists for a given instance.

In [118]:
class P:
    z = "Hello" # class variable
    
    def set_p(self):
        self.x = "Class P" # instance variable
    
    def print_p(self):
        print(self.x)

class C(P):
    def set_c(self):
        self.x = "Class C"
    
    def print_c(self):
        print(self.x)

In [119]:
c = C()
c.set_p() # instance variable x is defined for super class
c.print_p()
c.print_c() # x is not yet created for class C since set_c is not called

Class P
Class P


In [120]:
c.set_c()
c.print_p()
c.print_c()

Class C
Class C


#### class variables are inherited

In [121]:
c.z, C.z, P.z

('Hello', 'Hello', 'Hello')

In [122]:
C.z = "Bonjour" # this assignment creates a class variable 'z' for C
c.z, C.z, P.z

('Bonjour', 'Bonjour', 'Hello')

In [123]:
c.z = "Cia" # this assignment creates an instance variable 'z' for instance of C
c.z, C.z, P.z

('Cia', 'Bonjour', 'Hello')

### all together

In [124]:
class Circle(Shape):
    pi = 3.14159
    all_circles = []
    
    def __init__(self, r=1, x=0, y=0):
        super().__init__(x, y)
        self.radius = r
        Circle.all_circles.append(self)
    
    @classmethod
    def total_area(cls): # cls is reference to the class
        area = 0
        for circle in cls.all_circles:
            area += cls.circle_area(circle.radius)
        return area
    
    @staticmethod
    def circle_area(radius):
        return Circle.pi * radius * radius

In [125]:
c1 = Circle()
c1.radius, c1.x, c1.y

(1, 0, 0)

In [126]:
c2 = Circle(2, 1, 1)
c2.radius, c2.x, c2.y

(2, 1, 1)

In [127]:
Circle.all_circles

[<__main__.Circle at 0x23ca15a3550>, <__main__.Circle at 0x23ca25ad2e0>]

In [128]:
[c1, c2]

[<__main__.Circle at 0x23ca15a3550>, <__main__.Circle at 0x23ca25ad2e0>]

#### access class method through class or instance

In [129]:
Circle.total_area()

15.70795

In [130]:
c2.total_area()

15.70795

#### access static method through class or instance

In [131]:
Circle.circle_area(c1.radius)

3.14159

In [132]:
c1.circle_area(c1.radius)

3.14159

### private variables and private methods
* *private variable or private method* is one that cannot be seen outside the methods of the class in which it is defined.
* helps us to deny access to important or delicate part of an object's implementation.
* prevents name clashes that occur due to inheritance.

$Syntax$: any variable or method that ***starts with*** `__` (double underscore or *dunders*)

In [133]:
class Mine:
    def __init__(self):
        self.x = 2
        self.__y = 3 # private variable
    
    def print_y(self):
        print(self.__y) # accessed anywhere inside class

In [134]:
m = Mine()
print(m.x)

2


In [135]:
print(m.__y)

AttributeError: 'Mine' object has no attribute '__y'

In [136]:
m.print_y()

3


#### how python achieves private variables and methods (name mangling)?
* `_classname__variable` is how private variables are internally represented.
* so accessing `__variable` becomes invalid and throws `AttributeError`.

In [138]:
print(dir(m)) # the first one is '_Mine__y'

['_Mine__y', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'print_y', 'x']


#### hack private variables

In [139]:
m._Mine__y

3

In [140]:
m._Mine__y = 25
m.print_y()

25


### @property (flexible instance variables)
* getters, setters

In [141]:
class Temperature:
    def __init__(self):
        self._temp_fahr = 0 # internally stores the temperate in fahrenheit.
    
    @property # getter
    def celcius(self):
        return (self._temp_fahr - 32) * 5 / 9
    
    @celcius.setter # setter
    def celcius(self, cel):
        self._temp_fahr = cel * 9 / 5 + 32

In [142]:
t = Temperature()
t._temp_fahr

0

In [143]:
t.celcius

-17.77777777777778

In [144]:
t.celcius = 34
t._temp_fahr

93.2

In [145]:
t.celcius

34.0

#### exercise

In [146]:
class Rectangle:
    def __init__(self):
        self.__dimension = 0
    
    @property
    def dimension(self):
        return self.__dimension
    
    @dimension.setter
    def dimension(self, value):
        if value < 0:
            raise ValueError('Dimension of Rectangle can\'t be negative.')
        self.__dimension = value

In [147]:
r = Rectangle()
r.dimension

0

In [148]:
r.dimension = 4
r.dimension

4

In [149]:
r.dimension = -3

ValueError: Dimension of Rectangle can't be negative.

### destructors and memory management
* unlike C++, creating and calling a destructor is not necessary to ensure that the memory used by instance is freed.
* Python provides automatic memory management through a reference-counting mechanism.
* if number of references to an instance becomes 0, the memory is reclaimed.
* *we never need to define a destructor*.

* if we need to deallocate external resource explicitly, best practice is to use context manager.

***Next: Multiple Inheritance, Namespaces***